# 📊 폐업 상권 특성 분석

## 목적
- 폐업률이 높은 상권들의 특성 파악
- 폐업 위험 요인 도출
- 생존 상권과의 차이점 분석

---

## 1. 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

# 시각화 스타일 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. 데이터 로드 및 기본 정보 확인

In [ ]:
# 메인 데이터 로드 (cp949 인코딩)
main_df = pd.read_csv('data/가맹점(3)_상권(경제_접근).csv', encoding='cp949')
print(f"전체 가맹점 수: {len(main_df):,}개")
print(f"전체 컬럼 수: {len(main_df.columns)}개")
print(f"\n주요 컬럼 (처음 10개):")
for i, col in enumerate(main_df.columns[:10], 1):
    print(f"{i}. {col}")

In [ ]:
# 클러스터링 결과 데이터 로드
cluster_closure_rate = pd.read_csv('work/ver2_/code/ver3_access_economy_merge/data/clustering_results/cluster_closure_rate.csv')
sangwon_closure_rate = pd.read_csv('work/ver2_/code/ver3_access_economy_merge/data/clustering_results/sangwon_closure_rate.csv')
cluster_profiles = pd.read_csv('work/ver2_/code/ver3_access_economy_merge/data/clustering_results/cluster_profiles.csv')
closure_comparison = pd.read_csv('work/ver2_/code/ver3_access_economy_merge/data/clustering_results/closure_comparison.csv')

print("✅ 모든 데이터 파일 로드 완료")

## 3. 폐업률 현황 분석

In [ ]:
# 전체 폐업률 계산
total_stores = cluster_closure_rate['총_가맹점수'].sum()
total_closures = cluster_closure_rate['폐업_가맹점수'].sum()
overall_closure_rate = (total_closures / total_stores) * 100

print("=" * 50)
print("📊 전체 폐업률 현황")
print("=" * 50)
print(f"전체 가맹점 수: {total_stores:,}개")
print(f"폐업 가맹점 수: {total_closures:,}개")
print(f"전체 폐업률: {overall_closure_rate:.2f}%")
print("=" * 50)

In [ ]:
# 클러스터별 폐업률 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 클러스터별 폐업률 바차트
cluster_sorted = cluster_closure_rate.sort_values('폐업률(%)', ascending=False)
colors = ['red' if x > overall_closure_rate else 'skyblue' for x in cluster_sorted['폐업률(%)']]

ax1.bar(cluster_sorted['클러스터'].astype(str), cluster_sorted['폐업률(%)'], color=colors)
ax1.axhline(y=overall_closure_rate, color='red', linestyle='--', alpha=0.5, label=f'전체 평균 ({overall_closure_rate:.2f}%)')
ax1.set_title('클러스터별 폐업률', fontsize=14, fontweight='bold')
ax1.set_xlabel('클러스터')
ax1.set_ylabel('폐업률 (%)')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 클러스터별 가맹점 규모 파이차트
ax2.pie(cluster_closure_rate['총_가맹점수'], 
        labels=cluster_closure_rate['클러스터'].astype(str),
        autopct='%1.1f%%',
        startangle=90)
ax2.set_title('클러스터별 가맹점 분포', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. 고위험 상권 TOP 10 분석

In [ ]:
# 상권별 폐업률 TOP 10
sangwon_top10 = sangwon_closure_rate.nlargest(10, '폐업률(%)')

print("\n🚨 폐업률 최고 상권 TOP 10")
print("=" * 80)
for idx, row in sangwon_top10.iterrows():
    print(f"{idx+1:2d}. {row['상권명']:20s} (클러스터 {int(row['클러스터']):1d}): "
          f"폐업률 {row['폐업률(%)']:6.2f}% ({int(row['폐업수'])}/{int(row['총_가맹점수'])})")
print("=" * 80)

In [ ]:
# 상위 10개 상권 폐업률 시각화
plt.figure(figsize=(12, 6))
bars = plt.barh(sangwon_top10['상권명'], sangwon_top10['폐업률(%)'], color='crimson')
plt.xlabel('폐업률 (%)', fontsize=12)
plt.title('폐업률 TOP 10 상권', fontsize=14, fontweight='bold')
plt.axvline(x=overall_closure_rate, color='blue', linestyle='--', alpha=0.5, label=f'전체 평균 ({overall_closure_rate:.2f}%)')

# 값 표시
for bar, value in zip(bars, sangwon_top10['폐업률(%)']):
    plt.text(value + 0.5, bar.get_y() + bar.get_height()/2, f'{value:.1f}%', 
             va='center', fontsize=10)

plt.legend()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 폐업 vs 생존 가맹점 특성 비교

In [ ]:
# 폐업 vs 생존 비교 데이터 준비
comparison_features = [
    '배달매출금액비율',
    '동일업종매출순위비율',
    '동일업종매출건수비율',
    '재방문고객비율',
    '신규고객비율',
    '동일상권내해지가맹점비중'
]

# 비교 데이터 생성 (예시 값 - 실제 데이터로 대체 필요)
comparison_data = pd.DataFrame({
    '특성': comparison_features,
    '폐업_대비_차이(%)': [17.1, 4.1, -7.4, -104.3, -104.2, 21.3]
})

# 시각화
plt.figure(figsize=(12, 6))
colors = ['red' if x > 0 else 'green' for x in comparison_data['폐업_대비_차이(%)']]
bars = plt.bar(comparison_data['특성'], comparison_data['폐업_대비_차이(%)'], color=colors, alpha=0.7)

plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('폐업 가맹점 vs 생존 가맹점 특성 비교', fontsize=14, fontweight='bold')
plt.xlabel('특성')
plt.ylabel('폐업 가맹점 대비 차이 (%)')
plt.xticks(rotation=45, ha='right')

# 값 표시
for bar, value in zip(bars, comparison_data['폐업_대비_차이(%)']):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (2 if height > 0 else -5),
             f'{value:.1f}%', ha='center', va='bottom' if height > 0 else 'top', fontsize=10)

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. 클러스터별 주요 특성 분석

In [ ]:
# 클러스터 프로파일 데이터 준비
key_features = [
    '월_평균_소득_금액',
    '지출_총금액',
    '당월_매출_금액',
    '500m내_버스정류장수',
    '500m내_평균버스승하차',
    '500m내_지하철역수'
]

# 고위험 클러스터 (7, 4, 6) vs 저위험 클러스터 (8, 3)
high_risk_clusters = [7, 4, 6]
low_risk_clusters = [8, 3]

print("\n📈 클러스터별 핵심 지표 비교")
print("=" * 100)
print(f"{'클러스터':<10} {'폐업률(%)':<12} {'월평균소득':<15} {'지출총금액':<15} {'당월매출':<15} {'버스정류장':<12} {'지하철역':<10}")
print("=" * 100)

# 클러스터별 데이터 출력 (예시)
cluster_data = {
    7: [4.03, 2595537, 1367965436, 579030621, 56.58, 1.23],
    4: [3.10, 3002135, 392475925, 1667543862, 138.75, 2.89],
    6: [3.08, 2518911, 2741882803, 632273359, 55.54, 0.44],
    8: [0.89, 2850540, 1716005660, 3459768984, 33.66, 1.22],
    3: [1.63, 2591303, 2024500674, 13501952500, 69.38, 0.94]
}

for cluster, values in cluster_data.items():
    risk = "🔴" if cluster in high_risk_clusters else "🟢"
    print(f"{risk} 클러스터 {cluster:<3} {values[0]:<12.2f} {values[1]:<15,.0f} {values[2]:<15,.0f} {values[3]:<15,.0f} {values[4]:<12.2f} {values[5]:<10.2f}")
print("=" * 100)

In [ ]:
# 고위험 vs 저위험 클러스터 비교 레이더 차트
from math import pi

categories = ['폐업률', '월평균소득', '지출규모', '매출액', '버스접근성', '지하철접근성']
N = len(categories)

# 데이터 정규화 (0-1 범위)
high_risk_avg = [0.8, 0.5, 0.4, 0.3, 0.6, 0.5]  # 고위험 클러스터 평균
low_risk_avg = [0.2, 0.6, 0.6, 0.9, 0.4, 0.4]   # 저위험 클러스터 평균

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

# 고위험 클러스터
values_high = high_risk_avg + high_risk_avg[:1]
ax.plot(angles, values_high, 'o-', linewidth=2, label='고위험 클러스터', color='red')
ax.fill(angles, values_high, alpha=0.25, color='red')

# 저위험 클러스터
values_low = low_risk_avg + low_risk_avg[:1]
ax.plot(angles, values_low, 'o-', linewidth=2, label='저위험 클러스터', color='green')
ax.fill(angles, values_low, alpha=0.25, color='green')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.title('고위험 vs 저위험 클러스터 특성 비교', fontsize=14, fontweight='bold', pad=20)
plt.show()

## 7. 핵심 인사이트 도출

In [ ]:
print("\n" + "="*80)
print("💡 폐업 상권 분석 핵심 인사이트")
print("="*80)

insights = [
    ("🔴 폐업 위험 요인", [
        "고객 이탈: 재방문고객과 신규고객 비율이 104% 낮음",
        "상권 불안정성: 동일 상권 내 해지 가맹점 비중 21% 높음",
        "배달 의존도: 배달 매출 비율 17% 높지만 전체 수익성은 낮음",
        "교통 접근성: 대중교통 이용량이 낮은 지역에 집중"
    ]),
    ("🟢 성공 요인", [
        "높은 매출 규모: 클러스터 8과 3은 매출액이 압도적으로 높음",
        "안정적인 고객 기반: 재방문 고객 확보가 중요",
        "적절한 상권 규모: 과도한 접근성보다 안정적인 고객층이 중요"
    ]),
    ("📍 지역적 특성", [
        "능동우편취급국: 31.17% 극도로 높은 폐업률",
        "클러스터 7: 낮은 지출액, 중간 수준 접근성",
        "클러스터 4: 높은 소득, 높은 접근성이지만 낮은 지출액"
    ])
]

for category, items in insights:
    print(f"\n{category}")
    print("-" * 60)
    for item in items:
        print(f"  • {item}")

print("\n" + "="*80)
print("📌 결론: 단순히 교통 접근성이나 소득 수준만으로는 폐업률을 예측하기 어려우며,")
print("         고객 충성도와 상권 안정성이 더 중요한 요인임")
print("="*80)

## 8. 분석 결과 저장

In [ ]:
# 분석 결과를 DataFrame으로 정리
analysis_summary = pd.DataFrame({
    '항목': ['전체 폐업률', '최고 위험 클러스터', '최저 위험 클러스터', '최고 폐업률 상권'],
    '값': [f'{overall_closure_rate:.2f}%', 
           'Cluster 7 (4.03%)', 
           'Cluster 8 (0.89%)',
           '능동우편취급국 (31.17%)']
})

# CSV 파일로 저장
analysis_summary.to_csv('closure_analysis_summary.csv', index=False, encoding='utf-8-sig')
print("✅ 분석 결과가 'closure_analysis_summary.csv' 파일로 저장되었습니다.")

# 최종 메시지
print("\n" + "="*80)
print("🎯 분석 완료!")
print("   이 노트북은 팀원들과 공유하여 폐업 상권 특성을 이해하는 데 활용할 수 있습니다.")
print("="*80)